# Imports

In [ ]:
import pandas as pd
from hbt_tools import utils_hbt as uh
from pathlib import Path

# Functions

In [ ]:
def poleni(h, B, mu):
    return 2/3 * mu * (2 * 9.81)**(1/2) * B * (h)**(3/2)
#-------------------------------------------------------------------------------

def sum_per_period(periods, ts, date_format, idx_col=0):
    df_res = pd.DataFrame(columns=['von', 'bis', 'summe'])
    for i, p in enumerate(periods):
        date_start = pd.to_datetime(p[0], format=date_format)
        date_start = date_start.tz_localize('Europe/Zurich')
        date_end = pd.to_datetime(p[1], format=date_format)
        date_end = date_end.tz_localize('Europe/Zurich')

        ts_period = ts[(ts.index>=date_start) & (ts.index<=date_end)]
        df_temp = pd.DataFrame({'von': [p[0]], 'bis': [p[1]],
            'summe': [ts_period.iloc[:,idx_col].sum()]})
        
        df_res = pd.concat((df_res, df_temp))
    
    return df_res
#-------------------------------------------------------------------------------

def filter_ts(ts, date_from, date_to):
    ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

# Globals

In [ ]:
date_format = '%d.%m.%Y %H:%M'
date_start = (pd.to_datetime('28.09.2021 10:00', format=date_format)
    .tz_localize('Europe/Zurich'))
date_end = (pd.to_datetime('01.10.2022 00:00', format=date_format)
    .tz_localize('Europe/Zurich'))

ezg_tot = 2.97 # [ha]
ab_bw = 0.8 # [-]

periods = [
    ("28.09.2021 14:22", "01.12.2021 08:49"),
    ("01.12.2021 08:49", "08.12.2021 11:04"),
    ("08.12.2021 11:04", "29.12.2021 18:13"),
    ("29.12.2021 18:13", "04.01.2022 19:56"),
    ("07.02.2022 10:06", "01.04.2022 13:42"),
    ("01.04.2022 13:42", "26.04.2022 16:58"),
    ("26.04.2022 16:58", "23.05.2022 22:12"),
    ("23.05.2022 22:12", "01.06.2022 13:34"),
    ("02.06.2022 05:33", "07.06.2022 23:22"),
    ("07.06.2022 23:22", "22.06.2022 23:09"),
    ("22.06.2022 23:09", "24.06.2022 16:48"),
    ("24.06.2022 16:48", "04.07.2022 09:01"),
    ("04.07.2022 09:01", "04.07.2022 23:05"),
    ("04.07.2022 23:05", "23.07.2022 02:53"),
    ("23.07.2022 02:53", "15.08.2022 19:26"),
    ("15.08.2022 19:26", "19.08.2022 07:28"),
    ("22.09.2022 10:29", "28.09.2022 19:49"),
    ("28.09.2022 19:49", "08.10.2022 20:49"),
    ("08.10.2022 20:49", "04.11.2022 02:18"),
    ("04.11.2022 12:33", "05.11.2022 12:13"),
    ("05.11.2022 12:13", "10.12.2022 06:44"),
    ("10.12.2022 06:44", "24.12.2022 03:13"),
    ("24.12.2022 03:13", "26.12.2022 19:20"),
    ("26.01.2023 13:39", "12.03.2023 03:31"),
    ("12.03.2023 03:31", "25.03.2023 01:04"),
    ("25.03.2023 01:04", "03.04.2023 14:47"),
    ("03.04.2023 15:04", "24.04.2023 05:35"),
    ("24.04.2023 05:35", "01.05.2023 21:06"),
    ("01.05.2023 21:06", "10.05.2023 03:28")
]

date_from = periods[0][0]
date_to = periods[-1][-1]

# Regen

In [ ]:
path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Regendaten\LUZ_2020 bis 2023_data.csv")
ts_rain = uh.read_raindata_meteoschweiz(path_rain)

# Filter rain
ts_rain = filter_ts(ts_rain, date_start, date_end)

In [ ]:
df_rain_periods = sum_per_period(periods, ts_rain, date_format)
df_rain_periods

In [ ]:
tot_rain = ts_rain['rain'].sum()
print(f'Tot. Regen: {tot_rain:0.1f} mm')

vol_zu_rain = tot_rain * ezg_tot * 10
print(f'Tot. Zufluss (Regen): {vol_zu_rain:0.2f} m3')

# Zulauf

In [ ]:
id = 465
h_zu = uh.hbtdb_get_data(id, date_from, date_to, date_format)
h_zu.index = pd.to_datetime(h_zu.index, utc=True)
h_zu = h_zu - 105 # cm, 1.05 Messwert bei ansprigngen des Wehres/Entlastung (Christian)

# Convert Zulauf from h to Q (Poleni)
B = 2.75 # der Wert ist aus dem Schalungsplan
mu = 0.6
q_zu = poleni(B=B, mu=mu, h=h_zu/100) * 1000 # l/s
q_zu.fillna(0, inplace=True)

# Compute volume per time-step
q_zu['delta_t'] = q_zu.index
delta_t = q_zu['delta_t'] - q_zu['delta_t'].shift()
q_zu['delta_t'] = delta_t
q_zu['volumen'] = q_zu['wert'] * q_zu['delta_t'].dt.total_seconds()

In [ ]:
# Check dts
dt_min = delta_t.unique() / (1e9*60)
dt_min

In [ ]:
df_zufluss = sum_per_period(periods, q_zu, date_format, 2)
df_zufluss

In [ ]:
tot_vol_zu = filter_ts(q_zu, date_start, date_end)['volumen'].sum() / 1e3
print(f'Zufluss Vol: {tot_vol_zu:0.3f} m3')

# Ablauf

In [ ]:
id = 466
q_ab = uh.hbtdb_get_data(id, date_from, date_to, date_format)
q_ab.index = pd.to_datetime(q_ab.index, utc=True)

# Compute volume per time-step
q_ab['delta_t'] = q_ab.index
delta_t = q_ab['delta_t'] - q_ab['delta_t'].shift()
q_ab['delta_t'] = delta_t
q_ab['volumen'] = q_ab['wert'] * q_ab['delta_t'].dt.total_seconds()

In [ ]:
df_abfluss = sum_per_period(periods, q_ab, date_format, 2)
df_abfluss

In [ ]:
tot_vol_ab = filter_ts(q_ab, date_start, date_end)['volumen'].sum() / 1e3
print(f'Abfluss Vol: {tot_vol_ab:0.3f} m3')